In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
csv_path = os.path.join("..", "..", "data","LINKED_DATA", "TSR_EHR", "TSR_12_CLEANED.csv")
tsr_12 = pd.read_csv(csv_path)
tsr_12.head()

,height_nm,weight_nm,edu_id,pro_id,opc_id,ih_fl,ivtpamg_nm,hospitalised_time,nivtpa_id,nivtpa1_fl,...,nihs_9_out,nihs_10_out,nihs_11_out,total_out,SexName,Age,mrs_tx_1,mrs_tx_3,mrs_tx_6,mrs_tx_12
0,153.0,62.0,3,1,3,0,0.0,8.000,0,999,...,0,1,0,4,0,67.0,1,1,1,1
1,152.0,62.0,3,1,2,0,0.0,4.000,0,999,...,0,0,0,1,0,69.0,1,0,0,0
2,148.0,56.0,2,1,2,0,0.0,5.000,0,999,...,0,0,0,2,0,71.0,0,0,0,0
3,152.0,56.0,4,1,2,0,0.0,3.000,1,0,...,0,0,0,0,0,71.0,0,0,0,0
4,170.0,87.8,5,8,3,0,0.0,8.491,0,999,...,0,0,2,2,1,59.0,5,5,5,4


In [3]:
tsr_12_input = tsr_12.drop(["mrs_tx_12"], axis=1)
tsr_12_input[tsr_12_input == "N"] = 0
tsr_12_input[tsr_12_input == "Y"] = 1
tsr_12_input = tsr_12_input.astype("float64")
tsr_12_input = np.array(tsr_12_input.values)

tsr_12_input_nomrs = tsr_12.drop(["mrs_tx_12", "mrs_tx_6", "mrs_tx_3", "mrs_tx_1"], axis=1)
tsr_12_input_nomrs[tsr_12_input_nomrs == "N"] = 0
tsr_12_input_nomrs[tsr_12_input_nomrs == "Y"] = 1
tsr_12_input_nomrs = tsr_12_input_nomrs.astype("float64")
tsr_12_input_nomrs = np.array(tsr_12_input_nomrs.values)

# 6 classes

In [4]:
tsr_12_output = tsr_12.mrs_tx_12
tsr_12_output = tsr_12_output.astype("float64")
tsr_12_output = np.array(tsr_12_output.values)

## SVM

In [5]:
svc = CalibratedClassifierCV(LinearSVC(penalty = "l2", dual=False, loss = "squared_hinge", C = 1, multi_class = "ovr", 
                                       random_state = 19)) 
svc_scores = cross_val_score(svc,tsr_12_input,tsr_12_output,cv = 10,scoring='accuracy')
print(svc_scores)
print(svc_scores.mean(), svc_scores.std())

C:\Users\Jacky C\Anaconda3\envs\tsr_ml\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[0.53571429 0.46785714 0.525      0.47142857 0.57857143 0.51428571
 0.51785714 0.61290323 0.60215054 0.5734767 ]
0.5399244751664107 0.04804944114803709


In [6]:
svc.fit(tsr_12_input,tsr_12_output)
svc_predict =svc.predict_proba(tsr_12_input)
print(svc_predict)

[[0.27280328 0.42404413 0.1237437  0.05440595 0.09521942 0.02978351]
 [0.52820876 0.29301935 0.07158601 0.04158655 0.05910254 0.00649679]
 [0.52076478 0.2748044  0.06471258 0.03270939 0.1015961  0.00541276]
 ...
 [0.5157067  0.27182012 0.12339087 0.06166778 0.0227455  0.00466904]
 [0.00079225 0.12795276 0.14299713 0.07608717 0.12517148 0.52699921]
 [0.34600438 0.42162633 0.084594   0.09788345 0.04116082 0.00873103]]


In [7]:
svc_pred = cross_val_predict(svc,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, svc_pred)

C:\Users\Jacky C\Anaconda3\envs\tsr_ml\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


array([[456, 154,   2,   1,   3,   0],
       [177, 716,  12,   3,   8,   3],
       [ 22, 274,  14,  17,  22,  12],
       [  6, 174,  18,  12,  52,  14],
       [  8,  94,  15,  11, 108,  94],
       [  6,  24,   4,   6,  51, 204]], dtype=int64)

In [ ]:
svc1 = CalibratedClassifierCV(LinearSVC(penalty = "l2", dual=False, loss = "squared_hinge", C = 1, multi_class = "ovr", 
                                       random_state = 19)) 
svc_scores1 = cross_val_score(svc1,tsr_12_input_nomrs,tsr_12_output,cv = 10,scoring='accuracy')
print(svc_scores1)
print(svc_scores1.mean(), svc_scores1.std())

In [ ]:
svc1.fit(tsr_12_input_nomrs,tsr_12_output)
svc_predict1 =svc1.predict_proba(tsr_12_input_nomrs)
print(svc_predict1)

In [ ]:
svc_pred1 = cross_val_predict(svc,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, svc_pred1)

## RF

In [ ]:
rf = CalibratedClassifierCV(RandomForestClassifier(criterion = "gini", n_estimators = 15, bootstrap=True, random_state = 19,
                                                  max_features = 0.8)) 
rf_scores = cross_val_score(rf,tsr_12_input,tsr_12_output,cv = 10,scoring='accuracy')
print(rf_scores)
print(rf_scores.mean(), rf_scores.std())

In [ ]:
rf.fit(tsr_12_input,tsr_12_output)
rf_predict =rf.predict_proba(tsr_12_input)
print(rf_predict)

In [ ]:
rf_pred = cross_val_predict(rf,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, rf_pred)

In [ ]:
rf1 = CalibratedClassifierCV(RandomForestClassifier(criterion = "gini", n_estimators = 15, bootstrap=True, random_state = 19,
                                                  max_features = 0.8)) 
rf_scores1 = cross_val_score(rf1,tsr_12_input_nomrs,tsr_12_output,cv = 10,scoring='accuracy')
print(rf_scores1)
print(rf_scores1.mean(), rf_scores1.std())

In [ ]:
rf1.fit(tsr_12_input_nomrs,tsr_12_output)
rf_predict1 =rf1.predict_proba(tsr_12_input_nomrs)
print(rf_predict1)

In [ ]:
rf_pred1 = cross_val_predict(rf1,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, rf_pred1)

## XGBoost

In [ ]:
xgb = CalibratedClassifierCV(XGBClassifier(booster = "gbtree", objective="multi:softprob", eval_metric = "auc", 
                                            use_label_encoder = False, random_state = 19)) 
xgb_scores = cross_val_score(xgb,tsr_12_input,tsr_12_output,cv = 10)
print(xgb_scores)
print(xgb_scores.mean(), xgb_scores.std())

In [ ]:
xgb.fit(tsr_12_input,tsr_12_output)
xgb_predict =xgb.predict_proba(tsr_12_input)
print(xgb_predict)

In [ ]:
xgb_pred = cross_val_predict(xgb,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, xgb_pred)

In [ ]:
xgb1 = CalibratedClassifierCV(XGBClassifier(booster = "gbtree", objective="multi:softprob", eval_metric = "auc", 
                                            use_label_encoder = False, random_state = 19)) 
xgb_scores1 = cross_val_score(xgb1,tsr_12_input_nomrs,tsr_12_output,cv = 10)
print(xgb_scores1)
print(xgb_scores1.mean(), xgb_scores1.std())

In [ ]:
xgb1.fit(tsr_12_input_nomrs,tsr_12_output)
xgb_predict1 =xgb1.predict_proba(tsr_12_input_nomrs)
print(xgb_predict1)

In [ ]:
xgb_pred1 = cross_val_predict(xgb1,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, xgb_pred1)

# 2 classes

In [ ]:
tsr_12_output[(tsr_12_output == 0)|(tsr_12_output == 1)|(tsr_12_output == 2)] = 0
tsr_12_output[(tsr_12_output == 3)|(tsr_12_output == 4)|(tsr_12_output == 5)] = 1

## SVM

In [ ]:
svc2 = CalibratedClassifierCV(LinearSVC(penalty = "l2", dual=False, loss = "squared_hinge", C = 1, multi_class = "ovr", 
                                       random_state = 19)) 
svc_scores2 = cross_val_score(svc2,tsr_12_input,tsr_12_output,cv = 10,scoring='accuracy')
print(svc_scores2)
print(svc_scores2.mean(), svc_scores2.std())

In [ ]:
svc2.fit(tsr_12_input,tsr_12_output)
svc_predict2 =svc2.predict_proba(tsr_12_input)
print(svc_predict2)

In [ ]:
svc_pred2 = cross_val_predict(svc2,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, svc_pred2)

In [ ]:
svc3 = CalibratedClassifierCV(LinearSVC(penalty = "l2", dual=False, loss = "squared_hinge", C = 1, multi_class = "ovr", 
                                       random_state = 19)) 
svc_scores3 = cross_val_score(svc3,tsr_12_input_nomrs,tsr_12_output,cv = 10,scoring='accuracy')
print(svc_scores3)
print(svc_scores3.mean(), svc_scores3.std())

In [ ]:
svc3.fit(tsr_12_input_nomrs,tsr_12_output)
svc_predict3 =svc3.predict_proba(tsr_12_input_nomrs)
print(svc_predict3)

In [ ]:
svc_pred3 = cross_val_predict(svc3,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, svc_pred3)

## RF

In [ ]:
rf2 = CalibratedClassifierCV(RandomForestClassifier(criterion = "gini", n_estimators = 15, bootstrap=True, random_state = 19,
                                                  max_features = 0.8)) 
rf_scores2 = cross_val_score(rf2,tsr_12_input,tsr_12_output,cv = 10,scoring='accuracy')
print(rf_scores2)
print(rf_scores2.mean(), rf_scores2.std())

In [ ]:
rf2.fit(tsr_12_input,tsr_12_output)
rf_predict2 =rf2.predict_proba(tsr_12_input)
print(rf_predict2)

In [ ]:
rf_pred2 = cross_val_predict(rf2,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, rf_pred2)

In [ ]:
rf3 = CalibratedClassifierCV(RandomForestClassifier(criterion = "gini", n_estimators = 15, bootstrap=True, random_state = 19,
                                                  max_features = 0.8))  
rf_scores3 = cross_val_score(rf3,tsr_12_input_nomrs,tsr_12_output,cv = 10,scoring='accuracy')
print(rf_scores3)
print(rf_scores3.mean(), rf_scores3.std())

In [ ]:
rf3.fit(tsr_12_input_nomrs,tsr_12_output)
rf_predict3 =rf3.predict_proba(tsr_12_input_nomrs)
print(rf_predict3)

In [ ]:
rf_pred3 = cross_val_predict(rf3,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, rf_pred3)

## XGBoost

In [ ]:
xgb2 = CalibratedClassifierCV(XGBClassifier(booster = "gbtree", objective="binary:logistic", eval_metric = "auc", 
                                            use_label_encoder = False, random_state = 19))
xgb_scores2 = cross_val_score(xgb2,tsr_12_input,tsr_12_output,cv = 10)
print(xgb_scores2)
print(xgb_scores2.mean(), xgb_scores2.std())

In [ ]:
xgb2.fit(tsr_12_input,tsr_12_output)
xgb_predict2 =xgb2.predict_proba(tsr_12_input)
print(xgb_predict2)

In [ ]:
xgb_pred2 = cross_val_predict(xgb2,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, xgb_pred2)

In [ ]:
xgb3 = CalibratedClassifierCV(XGBClassifier(booster = "gbtree", objective="binary:logistic", eval_metric = "auc", 
                                            use_label_encoder = False, random_state = 19))
xgb_scores3 = cross_val_score(xgb3,tsr_12_input_nomrs,tsr_12_output,cv = 10)
print(xgb_scores3)
print(xgb_scores3.mean(), xgb_scores3.std())

In [ ]:
xgb3.fit(tsr_12_input_nomrs,tsr_12_output)
xgb_predict3 =xgb3.predict_proba(tsr_12_input_nomrs)
print(xgb_predict3)

In [ ]:
xgb_pred3 = cross_val_predict(xgb3,tsr_12_input,tsr_12_output,cv = 10)
confusion_matrix(tsr_12_output, xgb_pred3)

# Summary

## Mean & Std

In [ ]:
svc_mean = np.array([svc_scores.mean(), svc_scores.std(), svc_scores[0], svc_scores[1], svc_scores[2], svc_scores[3],
                     svc_scores[4], svc_scores[5], svc_scores[6], svc_scores[7], svc_scores[8], svc_scores[9]])
rf_mean = np.array([rf_scores.mean(), rf_scores.std(), rf_scores[0], rf_scores[1], rf_scores[2], rf_scores[3],rf_scores[4], 
                    rf_scores[5], rf_scores[6], rf_scores[7], rf_scores[8], rf_scores[9]])
xgb_mean = np.array([xgb_scores.mean(), xgb_scores.std(), xgb_scores[0], xgb_scores[1], xgb_scores[2], xgb_scores[3],
                     xgb_scores[4], xgb_scores[5], xgb_scores[6], xgb_scores[7], xgb_scores[8], xgb_scores[9]])

svc_mean1 = np.array([svc_scores1.mean(), svc_scores1.std(), svc_scores1[0], svc_scores1[1], svc_scores1[2], svc_scores1[3],
                     svc_scores1[4], svc_scores1[5], svc_scores1[6], svc_scores1[7], svc_scores1[8], svc_scores1[9]])
rf_mean1 = np.array([rf_scores1.mean(), rf_scores1.std(), rf_scores1[0], rf_scores1[1], rf_scores1[2], rf_scores1[3],rf_scores1[4], 
                    rf_scores1[5], rf_scores1[6], rf_scores1[7], rf_scores1[8], rf_scores1[9]])
xgb_mean1 = np.array([xgb_scores1.mean(), xgb_scores1.std(), xgb_scores1[0], xgb_scores1[1], xgb_scores1[2], xgb_scores1[3],
                     xgb_scores1[4], xgb_scores1[5], xgb_scores1[6], xgb_scores1[7], xgb_scores1[8], xgb_scores1[9]])

svc_mean2 = np.array([svc_scores2.mean(), svc_scores2.std(), svc_scores2[0], svc_scores2[1], svc_scores2[2], svc_scores2[3],
                     svc_scores2[4], svc_scores2[5], svc_scores2[6], svc_scores2[7], svc_scores2[8], svc_scores2[9]])
rf_mean2 = np.array([rf_scores2.mean(), rf_scores2.std(), rf_scores2[0], rf_scores2[1], rf_scores2[2], rf_scores2[3],rf_scores2[4], 
                    rf_scores2[5], rf_scores2[6], rf_scores2[7], rf_scores2[8], rf_scores2[9]])
xgb_mean2 = np.array([xgb_scores2.mean(), xgb_scores2.std(), xgb_scores2[0], xgb_scores2[1], xgb_scores2[2], xgb_scores2[3],
                     xgb_scores2[4], xgb_scores2[5], xgb_scores2[6], xgb_scores2[7], xgb_scores2[8], xgb_scores2[9]])

svc_mean3 = np.array([svc_scores3.mean(), svc_scores3.std(), svc_scores3[0], svc_scores3[1], svc_scores3[2], svc_scores3[3],
                     svc_scores3[4], svc_scores3[5], svc_scores3[6], svc_scores3[7], svc_scores3[8], svc_scores3[9]])
rf_mean3 = np.array([rf_scores3.mean(), rf_scores3.std(), rf_scores3[0], rf_scores3[1], rf_scores3[2], rf_scores3[3],rf_scores3[4], 
                    rf_scores3[5], rf_scores3[6], rf_scores3[7], rf_scores3[8], rf_scores3[9]])
xgb_mean3 = np.array([xgb_scores3.mean(), xgb_scores3.std(), xgb_scores3[0], xgb_scores3[1], xgb_scores3[2], xgb_scores3[3],
                     xgb_scores3[4], xgb_scores3[5], xgb_scores3[6], xgb_scores3[7], xgb_scores3[8], xgb_scores3[9]])

In [ ]:
tsr_12_mean = pd.DataFrame([svc_mean, rf_mean, xgb_mean,svc_mean1, rf_mean1, xgb_mean1, svc_mean2, rf_mean2, xgb_mean2,
                          svc_mean3, rf_mean3, xgb_mean3]).T
tsr_12_mean.index = ["Mean", "Std", "mean_1", "mean_2", "mean_3", "mean_4", "mean_5", "mean_6", "mean_7", "mean_8", "mean_9", "mean_10"]
tsr_12_mean.columns = ["svc", 'rf', 'xgb', "svc1", 'rf1', 'xgb1', 'svc2', 'rf2', 'xgb2', "svc3", 'rf3', 'xgb3']

In [ ]:
csv_save = os.path.join(".", "tsr_12_mean.csv")
tsr_12_mean.to_csv(csv_save, index = True)

## Predicted Probability

In [ ]:
svc_predict_0 = svc_predict[:, 0]
svc_predict_1 = svc_predict[:, 1]
svc_predict_2 = svc_predict[:, 2]
svc_predict_3 = svc_predict[:, 3]
svc_predict_4 = svc_predict[:, 4]
svc_predict_5 = svc_predict[:, 5]

rf_predict_0 = rf_predict[:, 0]
rf_predict_1 = rf_predict[:, 1]
rf_predict_2 = rf_predict[:, 2]
rf_predict_3 = rf_predict[:, 3]
rf_predict_4 = rf_predict[:, 4]
rf_predict_5 = rf_predict[:, 5]

xgb_predict_0 = xgb_predict[:, 0]
xgb_predict_1 = xgb_predict[:, 1]
xgb_predict_2 = xgb_predict[:, 2]
xgb_predict_3 = xgb_predict[:, 3]
xgb_predict_4 = xgb_predict[:, 4]
xgb_predict_5 = xgb_predict[:, 5]

svc_predict1_0 = svc_predict1[:, 0]
svc_predict1_1 = svc_predict1[:, 1]
svc_predict1_2 = svc_predict1[:, 2]
svc_predict1_3 = svc_predict1[:, 3]
svc_predict1_4 = svc_predict1[:, 4]
svc_predict1_5 = svc_predict1[:, 5]

rf_predict1_0 = rf_predict1[:, 0]
rf_predict1_1 = rf_predict1[:, 1]
rf_predict1_2 = rf_predict1[:, 2]
rf_predict1_3 = rf_predict1[:, 3]
rf_predict1_4 = rf_predict1[:, 4]
rf_predict1_5 = rf_predict1[:, 5]

xgb_predict1_0 = xgb_predict1[:, 0]
xgb_predict1_1 = xgb_predict1[:, 1]
xgb_predict1_2 = xgb_predict1[:, 2]
xgb_predict1_3 = xgb_predict1[:, 3]
xgb_predict1_4 = xgb_predict1[:, 4]
xgb_predict1_5 = xgb_predict1[:, 5]

svc_predict2_0 = svc_predict2[:, 0]
svc_predict2_1 = svc_predict2[:, 1]

rf_predict2_0 = rf_predict2[:, 0]
rf_predict2_1 = rf_predict2[:, 1]

xgb_predict2_0 = xgb_predict2[:, 0]
xgb_predict2_1 = xgb_predict2[:, 1]

svc_predict3_0 = svc_predict3[:, 0]
svc_predict3_1 = svc_predict3[:, 1]

rf_predict3_0 = rf_predict3[:, 0]
rf_predict3_1 = rf_predict3[:, 1]

xgb_predict3_0 = xgb_predict3[:, 0]
xgb_predict3_1 = xgb_predict3[:, 1]

In [ ]:
tsr_12_pred_prob = pd.DataFrame([svc_predict_0, svc_predict_1, svc_predict_2, svc_predict_3, svc_predict_4, svc_predict_5, 
                               rf_predict_0, rf_predict_1, rf_predict_2, rf_predict_3, rf_predict_4, rf_predict_5,
                               xgb_predict_0, xgb_predict_1, xgb_predict_2, xgb_predict_3, xgb_predict_4, xgb_predict_5,
                                svc_predict1_0, svc_predict1_1, svc_predict1_2, svc_predict1_3, svc_predict1_4, svc_predict1_5, 
                               rf_predict1_0, rf_predict1_1, rf_predict1_2, rf_predict1_3, rf_predict1_4, rf_predict1_5,
                               xgb_predict1_0, xgb_predict1_1, xgb_predict1_2, xgb_predict1_3, xgb_predict1_4, xgb_predict1_5,
                               svc_predict2_0, svc_predict2_1, rf_predict2_0, rf_predict2_1, xgb_predict2_0, xgb_predict2_1,
                               svc_predict3_0, svc_predict3_1, rf_predict3_0, rf_predict3_1, xgb_predict3_0, xgb_predict3_1]).T
tsr_12_pred_prob.columns = ["svc_predict_0", "svc_predict_1", "svc_predict_2", "svc_predict_3", "svc_predict_4", "svc_predict_5", 
                           "rf_predict_0", "rf_predict_1", "rf_predict_2", "rf_predict_3", "rf_predict_4", "rf_predict_5",
                           "xgb_predict_0", "xgb_predict_1", "xgb_predict_2", 'xgb_predict_3', "xgb_predict_4", "xgb_predict_5",
                           "svc_predict1_0", "svc_predict1_1", "svc_predict1_2", "svc_predict1_3", "svc_predict1_4", "svc_predict1_5", 
                           "rf_predict1_0", "rf_predict1_1", "rf_predict1_2", "rf_predict1_3", "rf_predict1_4", "rf_predict1_5",
                           "xgb_predict1_0", "xgb_predict1_1", "xgb_predict1_2", 'xgb_predict1_3', "xgb_predict1_4", "xgb_predict1_5",
                           "svc_predict2_0", "svc_predict2_1", "rf_predict2_0", "rf_predict2_1", "xgb_predict2_0", "xgb_predict2_1",
                           "svc_predict3_0", "svc_predict3_1", "rf_predict3_0", "rf_predict3_1", "xgb_predict3_0", "xgb_predict3_1"]

In [ ]:
csv_save2 = os.path.join(".", "tsr_12_pred_prob.csv")
tsr_12_pred_prob.to_csv(csv_save2, index = False)